In [1]:
import os
from datetime import timedelta
import cv2
from PIL import Image
import numpy as np
import torch

In [2]:
VIDEO_FILE = "./data/Vídeo 1 - Sassoeiros - ACP [BakzGDcTsVE].webm"
FRAME_FOLDER = "./data/Vídeo 1 - Sassoeiros - ACP [BakzGDcTsVE]-opencv/"
# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 10

In [3]:
from pickle import FRAME


def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    # clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    clip_duration = 5  # only get first 5 seconds
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s


def extract_video_frames(video_file):
    # make a folder by the name of the video file
    if not os.path.isdir(FRAME_FOLDER):
        os.mkdir(FRAME_FOLDER)
    # read the video file    
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(FRAME_FOLDER, f"frame{frame_duration_formatted}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

In [4]:
# Extracting video frames to FRAME_FOLDER
extract_video_frames(VIDEO_FILE)

## Object Detection

In [ ]:
# Yolo Inference
# PyTorch Hub Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5n - yolov5x6, custom

# Images
pandas_results = []
for img in os.listdir("./frames"):
  im = Image.open(f"./frames/{img}")
  # Inference
  results = model(im)
  results.save()
  pandas_results.append(results.pandas().xyxy[0])

## Team Recognition

## Player Identification